# YAKC - Model

## Import

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from IPython.display import display, Markdown
plt.style.use("seaborn-darkgrid")
pd.set_option('display.max_columns', None)  

DEBUG = False
SEED = 666

import os
for d in ['orig','data','output']: os.makedirs(d, exist_ok=True)


In [2]:
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import PCA

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

classifiers = {
    #"LR": LogisticRegression(),
    "DT(max_depth=3)": DecisionTreeClassifier(max_depth=3),
    "DT(max_depth=6)": DecisionTreeClassifier(max_depth=6),
    "DT(max_depth=9)": DecisionTreeClassifier(max_depth=9),
    "KNN": KNeighborsClassifier(n_neighbors = 5),
    "SVC": SVC(),
    "RFC": RandomForestClassifier(),
    "ADA": AdaBoostClassifier(),
}

In [21]:
models = []

# Naive Bayes
from sklearn.naive_bayes import GaussianNB
models.append( {"name": "NB", "ref": GaussianNB()} )

# k-nearest neighbor 
from sklearn.neighbors import KNeighborsClassifier
models.append( {"name": "KNN", "ref": KNeighborsClassifier()} )

# Logistic Regression
from sklearn.linear_model import LogisticRegression
models.append( {"name": "LogisticRegression", "ref": LogisticRegression(solver="lbfgs")} )

# Support Vector Machines
from sklearn.svm import SVC
models.append( {"name": "SVC", "ref": SVC(gamma="scale")} )

# Decision Trees
from sklearn.tree import DecisionTreeClassifier
models.append( {"name": "DT (pruned=4)", "ref": DecisionTreeClassifier(criterion='entropy', max_depth=4)} )

## Datasets

#### Train

In [4]:
df = pd.read_pickle("data/train.pkl")
print(df.shape)
df

(18735, 207)


,x000,x001,x002,x003,x004,x005,x006,x007,x008,x009,x010,x011,x012,x013,x014,x015,x016,x017,x018,x019,x020,x021,x022,x023,x024,x025,x026,x027,x028,x029,x030,x031,x032,x033,x034,x035,x036,x037,x038,x039,x040,x041,x042,x043,x044,x045,x046,x047,x048,x049,x050,x051,x052,x053,x054,x055,x056,x057,x058,x059,x060,x061,x062,x063,x064,x065,x066,x067,x068,x069,x070,x071,x072,x073,x074,x075,x076,x077,x078,x079,x080,x081,x082,x083,x084,x085,x086,x087,x088,x089,x090,x091,x092,x093,x094,x095,x096,x097,x098,x099,x100,x101,x102,x103,x104,x105,x106,x107,x108,x109,x110,x111,x112,x113,x114,x115,x116,x117,x118,x119,x120,x121,x122,x123,x124,x125,x126,x127,x128,x129,x130,x131,x132,x133,x134,x135,x136,x137,x138,x139,x140,x141,x142,x143,x144,x145,x146,x147,x148,x149,x150,x151,x152,x153,x154,x155,x156,x157,x158,x159,x160,x161,x162,x163,x164,x165,x166,x167,x168,x169,x170,x171,x172,x173,x174,x175,x176,x177,x178,x179,x180,x181,x182,x183,x184,x185,x186,x187,x188,x189,x190,x191,x192,x193,x194,x195,x196,x197,x198,x199,x200,x201,x202,x203,x204,x205,target
0,3,77.4395,1,0.8533,14.9130,7.4400,0.54,3.522070,0.000000,1043.82,0.720,7,1.1346,0.1733,0.0000,-0.318995,0.000256,1.210252,15.4300,1382.2740,0.720,0.000272,23.4832,0.001704,527.6685,0.0,1.1056,2.995991,0.046392,0.0000,0.1987,3.751083,0.0,3,253.17,0.2096,0.675,0.0,3,0.0178,0.2044,94.8683,2.295,2.925321,4.1667,1.1594,2.1226,60.000,1706.86,0.1853,0.043272,0.0000,0.2880,1,0.0,3,0.0000,0.900,-0.925930,3,0.000296,152.9706,3.459193,0.0652,1.915986,67.082,1.0,5,0.0,NaN,0.0356,0.0000,2.0,0.886684,0.0,0.306582,0.0000,0.0000,0.0000,6.12,3.0974,22.320,1.955933,0.000,0.0000,10.3810,0.0,39.1053,12.78,-0.879191,0.0356,0.1511,1.0790,1.1328,0.3118,4.0625,0.009304,0.00,-6.142256,5.76,0.4578,4,3,1,16.0654,4.5387,0.0000,1,0.0222,0.0000,0.000000,0.00,973.62,0.0,0.000000,0.8,1.1448,0.6088,0.0000,1,38.34,10.8933,3,0.443059,0.0000,5,0.0000,4.1176,4,0.188757,0.0000,0.000000,0.0000,1.1263,3,2.295,1.1497,0.000,0.00,6.66,35.0627,13.0509,1.1627,1,209.34,3.522070,1.955933,1397.0974,0,1931.4836,0.045495,-0.525758,0.0000,-1.219289,0.0000,4.5000,0,1.915986,0.0000,1.1807,67.0820,0.00,1.0713,19950.39,6.2253,4.742285,0.0,2,0.000,0.0000,13.293280,1,211.0660,-1.474187,3.081379,0.0933,2015,1.440,0.0000,-0.925930,0.000568,2,24.1959,0,1704,0.0000,0.0000,-1.330913,0.1911,5,-0.004427,1.1091,0.00,3.152907,2.925321,0.4432,0.99,21.6853,1.0718,3.0,152.9706,2.0,0.0,0.0,0,0.011252,1.0
1,3,12.5110,1,1.9778,25.1193,0.0000,0.00,-0.996876,0.000000,0.00,0.000,6,0.0000,0.0000,0.0000,2.109710,0.000000,-0.934238,23.0435,1457.9697,0.810,0.000220,0.0000,0.000000,0.0000,0.0,1.1103,0.111828,0.000000,0.0000,0.0347,0.415993,0.0,2,0.00,0.2995,0.495,0.0,3,0.0044,0.0000,0.0000,1.620,0.777478,1.6250,0.0000,0.0000,60.000,1343.77,0.2027,0.004672,0.0000,0.0480,1,0.0,2,0.0000,0.810,-4.497039,3,0.000052,0.0000,-2.646247,0.0636,0.403263,60.000,NaN,4,0.0,1.0,0.0000,0.0000,2.0,0.818036,0.0,0.304375,0.0000,0.0000,0.0000,4.95,0.0000,0.000,1.737248,0.000,0.0000,0.0000,0.0,11.3913,1.62,0.560984,0.0444,0.0000,1.1214,1.1552,0.0000,0.0000,0.176948,0.00,-0.055706,0.00,0.0133,4,3,1,0.0000,0.0000,0.0000,1,0.0089,0.0000,0.000000,0.00,105.12,0.0,0.000000,0.9,1.0523,0.1350,0.0000,1,0.00,0.0000,3,2.284246,0.0000,5,0.0000,5.0667,4,-1.008446,0.0000,0.000000,0.0000,1.1214,3,0.000,0.0000,0.000,0.00,1.17,58.4320,0.0000,0.0000,2,3981.33,-0.996876,1.737248,617.9989,0,0.0000,-0.543308,2.684732,0.0000,0.071503,0.0000,2.0000,0,0.403263,0.0000,1.2840,301.4963,0.00,87.5903,18405.81,56.7627,1.174342,0.0,2,0.000,0.0000,-1.718136,1,67.0820,-0.491866,1.766481,0.0000,2018,0.585,0.0000,-4.497039,0.000072,2,50.9762,0,1327,0.0000,0.0000,1.161308,0.6800,5,1.119686,1.1214,0.00,-2.594028,0.777478,0.0450,27.72,2.0853,0.0000,3.0,0.0000,2.0,0.0,0.0,0,0.000000,1.0
2,3,18.5793,1,0.4889,10.4918,0.9533,0.63,1.749067,0.000000,97.38,0.000,7,1.1494,0.0720,0.0000,0.646423,0.000124,-0.407236,11.0795,3928.3637,0.630,0.000576,15.4689,0.000000,211.0660,0.0,1.0945,2.798307,0.004328,0.0000,0.1120,-2.228379,0.0,4,486.99,0.1144,0.810,1.

In [5]:
df_score = pd.read_pickle("data/test.pkl")
print(df_score.shape)
df_score

(6265, 206)


,x000,x001,x002,x003,x004,x005,x006,x007,x008,x009,x010,x011,x012,x013,x014,x015,x016,x017,x018,x019,x020,x021,x022,x023,x024,x025,x026,x027,x028,x029,x030,x031,x032,x033,x034,x035,x036,x037,x038,x039,x040,x041,x042,x043,x044,x045,x046,x047,x048,x049,x050,x051,x052,x053,x054,x055,x056,x057,x058,x059,x060,x061,x062,x063,x064,x065,x066,x067,x068,x069,x070,x071,x072,x073,x074,x075,x076,x077,x078,x079,x080,x081,x082,x083,x084,x085,x086,x087,x088,x089,x090,x091,x092,x093,x094,x095,x096,x097,x098,x099,x100,x101,x102,x103,x104,x105,x106,x107,x108,x109,x110,x111,x112,x113,x114,x115,x116,x117,x118,x119,x120,x121,x122,x123,x124,x125,x126,x127,x128,x129,x130,x131,x132,x133,x134,x135,x136,x137,x138,x139,x140,x141,x142,x143,x144,x145,x146,x147,x148,x149,x150,x151,x152,x153,x154,x155,x156,x157,x158,x159,x160,x161,x162,x163,x164,x165,x166,x167,x168,x169,x170,x171,x172,x173,x174,x175,x176,x177,x178,x179,x180,x181,x182,x183,x184,x185,x186,x187,x188,x189,x190,x191,x192,x193,x194,x195,x196,x197,x198,x199,x200,x201,x202,x203,x204,x205
0,3,4.2241,1,0.0000,23.5068,29.6787,0.450,1.980838,0.163172,16295.85,0.00,5,1.0945,0.0533,0.1333,0.430486,0.000064,-0.552766,15.8210,30.8575,0.000,0.000000,3518.1899,0.000000,0.0000,0.000,0.0000,-2.669028,0.724260,11.5360,0.0000,1.363201,0.0000,2,0.00,0.4409,0.000,0.0,4,0.0000,0.0000,0.0000,0.00,0.292289,0.0000,0.0000,0.0000,90.000,990.09,0.0000,0.027496,0.0,0.0000,1,0.000,2,1.1065,0.72,-0.383887,3,0.000000,60.0000,-0.999931,0.0676,2.405041,67.0820,1.0,5,0.0,NaN,0.0133,522.5555,NaN,0.084792,0.0000,0.252765,0.0,0.1747,0.0000,0.00,0.0000,2.340,-4.421815,0.54,0.0000,0.0000,0.000000,44.1145,0.00,0.257967,0.0000,0.0711,1.1169,0.0000,0.0424,2.5000,0.000000,3671.37,-1.765610,1.44,1.1022,3,2,1,0.0000,0.0000,216.0818,1,0.0000,0.0,0.0,0.0,618.66,0.0,0.000216,2.4,0.0000,0.0000,10.9480,1,0.00,0.0000,3,-0.101281,0.0000,5,4.4667,0.0000,5,-0.099349,0.0000,0.000000,0.0,1.1216,3,0.000,0.0000,0.0,0.00,0.00,18.1667,14.0200,0.0000,2,0.00,1.980838,-4.421815,0.0000,0,3707.0339,0.255978,1.562013,1.0958,-0.211940,0.0000,0.0000,0,2.405041,0.1786,1.1440,0.0000,0.00,0.0000,1907.82,0.0000,2.883239,0.0000,2,58.005,0.0356,-2.337341,1,0.0000,0.412650,-4.120612,0.0000,2021,0.000,227.9023,-0.383887,0.000000,2,0.0000,0,995,0.0,0.0,-2.854167,1.2489,5,1.875138,0.0000,4.86,1.518522,0.292289,0.0000,0.99,19.4360,1.0699,3.0,0.0000,1.0,0.00,0.000,0,0.000000
1,3,0.0000,1,1.3244,6.6164,0.0000,0.540,-2.606239,0.000000,0.00,0.00,7,0.0000,0.0160,0.0000,-0.835329,0.000024,-0.561637,6.5965,9702.2157,0.765,0.000000,0.0000,0.000000,0.0000,0.000,0.0000,-0.999451,0.000000,0.0000,0.4960,1.520167,0.0000,3,2.07,0.0499,0.000,0.0,3,0.0000,0.0044,0.0000,0.00,-0.457013,7.2692,1.1671,0.0000,60.000,1465.89,0.0000,0.003376,0.0,0.0000,1,0.000,2,0.0000,905.58,1.763969,3,0.000676,0.0000,-1.396045,0.0560,0.625527,0.0000,1.0,4,0.0,0.0,0.0044,0.0000,2.0,0.974472,0.0000,-0.245821,0.0,0.0000,0.0000,0.00,0.0000,0.000,-2.442345,0.00,0.0000,0.0000,0.000000,5.2542,0.00,-1.137698,0.0000,0.0000,1.1557,0.0000,0.0000,1.2000,0.021360,0.00,-1.965061,0.54,0.0044,5,3,1,2.2000,0.0587,0.0000,1,0.1022,0.0,0.0,0.0,75.96,0.0,0.000000,1.3,1.0831,0.3147,0.0000,1,0.00,0.0000,4,-1.083037,0.0000,5,0.0000,0.0000,3,0.018753,0.0000,0.000000,0.0,1.1347,3,2.070,0.0000,0.0,0.00,15.21,14.8267,0.0000,0.0000,2,480.60,-2.606239,-2.442345,0.0000,0,0.0000,-1.329380,0.826567,0.0000,0.175797,0.0000,0.0000,0,0.625527,0.0000,1.2473,379.4733,0.00,3.3791,21925.62,13.6387,0.838948,0.0000,2,0.000,0.0000,-0.176897,1,90.0000,0.784310,0.165931,0.0000,2015,0.540,0.0000,1.763969,0.000000,2,34.8912,0,1456,0.0,0.0,2.020684,0.0133,4,-2.273555,1.1382,0.00,3.998988,-0.457013,0.0000,75.96,0.8120,1.0472,3.0,0.0000,2.0,0.00,0.000,0,0.000092
2,3,6.7518,1,0.0089,31.1296,47.9627,0.000,1.907240,0.000028,16537.50,0.00,6,1.1297,0.0000,0.0044,-0.335846,0.000000,-0.323353,21.2705,71.2133,0.630,0.000000,3490.4073,0.000000,0.0000,0.000,0.0000,0.923170,0.735000,0.0267,1.1227,2.840879,0.0000,3,0.00,0.4035,0.000,1.0,3,0.0000,0.0000,0.0000,0.00,6.34690

## Model

#### Preprocessing

In [6]:
df.fillna(0, inplace=True)
df['target'] = df['target'].astype(int)
df_score.fillna(0, inplace=True)

In [7]:
df_train, df_test = train_test_split(df, stratify=df.target, test_size=0.4, random_state=SEED)
df_train.shape, df_test.shape

((11241, 207), (7494, 207))

In [8]:
target = 'target'
features = df_train.columns.tolist()
features.remove(target)
len(features)

206

In [9]:
ss = StandardScaler()
X_train = ss.fit_transform(df_train[features])
y_train = df_train[target]

X_test = ss.fit_transform(df_test[features])
y_test = df_test[target]


#### Model Selection

Go through every classifier and quickly score it.

In [10]:
classifying_classifiers = False

In [11]:
if classifying_classifiers:
    for classifier in classifiers:
        model = classifiers[classifier]
        scores = cross_val_score(model, X_train, y_train, cv=10)
        print(classifier + ": " + str(scores.mean()) + ", " + str(scores.std()))


#### Choosing model for current output

In [12]:
classifier = "DT(max_depth=9)"

In [13]:
model = classifiers[classifier]

#### Hyperparameter Tuning

#### Dimensionality Reduction

In [14]:


#best_mean = 0
#for i in range(1, len(X_train)):
  #  pca = PCA(n_components=i)
 #   X_train_pca = pca.fit_transform(X_train)
 #   scores = cross_val_score(model, X_train_pca, y_train, cv=10)
 ##   if best_mean > scores.mean():
 #       break
#    best_mean = scores.mean()
 #   print(i + ": " + str(scores.mean()) + ", " + str(scores.std())) 


In [19]:
i=150
pca = PCA(n_components=i)
X_train_pca = pca.fit_transform(X_train)

scores = cross_val_score(model, X_train_pca, y_train, cv=10)
print(str(i) + ": " + str(scores.mean()) + ", " + str(scores.std())) 

scores = cross_val_score(model, X_train, y_train, cv=10)
print( "Base: " + str(scores.mean()) + ", " + str(scores.std())) 

150: 0.743971451166469, 0.011609014783534282
Base: 0.7844483194938711, 0.007994370772421586


#### Using Feature cut down from churn

In [22]:
print("Number of features starting:", len(features))

Number of features starting: 206


In [23]:
# list of our modules that can be used with RFECV
suitable_models = [model for model in models if model['name'].split()[0] in ['LogisticRegression','DT'] ]

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV

model = suitable_models[1]

rfecv = RFECV(estimator=model["ref"], step=1, cv=StratifiedKFold(2), scoring="f1")
rfecv.fit(X_train, y_train)
print("Optimal number of features : %d" % rfecv.n_features_)

# Plot number of features VS. cross-validation scores
plt.figure()
plt.title(f"{model['name']} with RFECV")
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score")
plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
plt.show()

In [170]:
# list of features selected
features_selected = np.array(features)[rfecv.support_].tolist()
features_selected

['Intl_Plan', 'VMail_Plan', 'Intl_Calls', 'CustServ_Calls', 'Total_Charge']

### Training - After Feature Selection

In [171]:
display(df_model.loc[:, features_selected + [target]].head(1))

# normalise 
X_train = scaler.fit_transform(df_train.loc[:,features_selected].values)
y_train = df_train.Churn.values
X_test = scaler.transform(df_test.loc[:,features_selected].values)
y_test = df_test.Churn.values

train_models()

,Intl_Plan,VMail_Plan,Intl_Calls,CustServ_Calls,Total_Charge,Churn
0,0,1,3,1,75.56,0



Model NB
	CV scores ['0.4912', '0.4074', '0.3333', '0.5200', '0.4074', '0.4727', '0.5263', '0.5263', '0.4151', '0.3214']
	mean=44.21% std=0.0729

Model KNN
	CV scores ['0.6939', '0.7200', '0.8000', '0.8302', '0.7083', '0.7200', '0.6939', '0.7755', '0.6667', '0.5714']
	mean=71.80% std=0.0693

Model LogisticRegression
	CV scores ['0.3684', '0.1667', '0.0976', '0.2703', '0.0625', '0.2703', '0.4186', '0.3636', '0.2439', '0.1667']
	mean=24.28% std=0.1131

Model SVC
	CV scores ['0.7083', '0.6800', '0.8235', '0.8235', '0.6957', '0.7347', '0.6809', '0.7660', '0.6809', '0.5581']
	mean=71.52% std=0.0742

Model DT (pruned=4)
	CV scores ['0.8400', '0.8235', '0.8627', '0.9057', '0.8400', '0.8627', '0.8627', '0.8846', '0.8400', '0.8846']
	mean=86.07% std=0.0242

Best Performing Model DT (pruned=4) with mean=86.07%


## Evaluation

In [1]:
model.fit(X_train, y_train)

NameError: name 'model' is not defined

In [2]:
y_pred = model.predict(X_test)

NameError: name 'model' is not defined

In [3]:
print(classification_report(y_test, y_pred))

NameError: name 'classification_report' is not defined

## Scoring
* Retain on all data and generate predictions on df_score

In [ ]:
ss = StandardScaler()
X_train = ss.fit_transform(df[features])
y_train = df[target]

X_test = ss.fit_transform(df_score[features])

In [ ]:
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
y_pred

In [ ]:
df_score['target'] = y_pred
df_score[['target']].reset_index().to_csv("output/solution.csv", index=False)